# PREDATA


In [1]:
import tensorflow as tf

# Ver dispositivos disponibles
print("Dispositivos disponibles:")
for device in tf.config.list_physical_devices():
    print(device)

# Ver si TensorFlow está usando GPU
if tf.test.is_gpu_available():
    print("\n✅ TensorFlow está usando la GPU.")
    print("GPU en uso:", tf.config.experimental.list_physical_devices('GPU'))
else:
    print("\n❌ TensorFlow NO está usando la GPU.")


Dispositivos disponibles:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

✅ TensorFlow está usando la GPU.
GPU en uso: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, False)  # Usar toda la memoria

In [ ]:
import numpy as np
import itertools
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Activation, Dense
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import os
import random
import shutil
import pandas as pd

## CREACION DE DATAFRAME

In [5]:
# Ruta al directorio donde están las imágenes
data_dir = "../data/morton14k"

# Definir reglas de clasificación
CLASS_RULES = {
    "chat": ["chat"],
    "email": ["email"],
    "voice": ["audio", "voipbuster"],
    "streaming": ["vimeo", "netflix", "spotify", "youtube", "video"],
    "filetransfer": ["sftp", "ftps", "files", "file"],
    "p2p": ["bittorrent"]
}

# Función para asignar una etiqueta según el nombre del archivo
def get_label(filename):
    for category, keywords in CLASS_RULES.items():
        if any(keyword in filename.lower() for keyword in keywords):
            return category
    return None  # Si no coincide con ninguna categoría

# Recorrer la carpeta y subcarpetas para extraer información
image_paths = []
labels = []

for root, _, files in os.walk(data_dir):  # Recorre todas las carpetas y archivos
    for file in files:
        if file.endswith(".png"):  # Solo archivos PNG
            label = get_label(file)
            if label:  # Solo añadimos imágenes con una categoría válida
                image_paths.append(os.path.join(root, file))  # Ruta completa
                labels.append(label)

# Crear DataFrame
df = pd.DataFrame({"image_path": image_paths, "label": labels})

# Mostrar las primeras filas para verificar
print(df.tail())
print(df["label"].value_counts().to_frame().reset_index().rename(columns={"index": "Etiqueta", "label": "Cantidad"}))


                                              image_path      label
86995  ../data/morton14k\f_youtubeHTML5_1_9880.bin_mo...  streaming
86996  ../data/morton14k\f_youtubeHTML5_1_9888.bin_mo...  streaming
86997  ../data/morton14k\f_youtubeHTML5_1_990.bin_mor...  streaming
86998  ../data/morton14k\f_youtubeHTML5_1_9959.bin_mo...  streaming
86999  ../data/morton14k\f_youtubeHTML5_1_9987.bin_mo...  streaming
       Cantidad  count
0          chat  14500
1         email  14500
2         voice  14500
3     streaming  14500
4  filetransfer  14500
5           p2p  14500


In [6]:
# Crear un diccionario para convertir etiquetas en números
label_map = {label: idx for idx, label in enumerate(df["label"].unique())}
df["label"] = df["label"].map(label_map)  # Convertimos a números

# Función para cargar imágenes y etiquetas
# Función para cargar imágenes y convertir etiquetas a one-hot encoding
def load_image_label(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)  # Convertir PNG a tensor
    img = tf.image.resize(img, (32, 32))  # Asegurar tamaño correcto
    img = img / 255.0  # Normalización [0,1]

    label = tf.one_hot(label, depth=6)  # Convertir etiqueta a one-hot
    return img, label


# Convertir el DataFrame en un Dataset de TensorFlow
dataset = tf.data.Dataset.from_tensor_slices((df["image_path"].values, df["label"].values))
dataset = dataset.map(load_image_label)

# Mostrar un ejemplo
for img, label in dataset.take(1):  
    print(img.shape, label.numpy())

(32, 32, 3) [1. 0. 0. 0. 0. 0.]


In [7]:
## SEPARACION DE TRAINING Y VALIDATION DATA

# Ver distribución de clases antes de dividir
print("Distribución de clases en el dataset original:\n")
print(df["label"].value_counts())

# Dividir en entrenamiento y validación asegurando la estratificación
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Verificar distribución después de dividir
print("\nDistribución de clases en el dataset de entrenamiento:")
print(train_df["label"].value_counts())

print("\nDistribución de clases en el dataset de validación:")
print(val_df["label"].value_counts())

# Crear datasets de TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((train_df["image_path"].values, train_df["label"].values))
val_dataset = tf.data.Dataset.from_tensor_slices((val_df["image_path"].values, val_df["label"].values))

# Aplicar la función de carga de imágenes
train_dataset = train_dataset.map(load_image_label)
val_dataset = val_dataset.map(load_image_label)

# Contar elementos en los datasets de TensorFlow
train_count = sum(1 for _ in train_dataset)
val_count = sum(1 for _ in val_dataset)

print(f"\nTotal imágenes en entrenamiento: {train_count}")
print(f"Total imágenes en validación: {val_count}")

# Mezclar, agrupar en batches y optimizar
batch_size = 64
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(AUTOTUNE)


Distribución de clases en el dataset original:

label
0    14500
1    14500
2    14500
3    14500
4    14500
5    14500
Name: count, dtype: int64

Distribución de clases en el dataset de entrenamiento:
label
1    11600
2    11600
5    11600
0    11600
4    11600
3    11600
Name: count, dtype: int64

Distribución de clases en el dataset de validación:
label
1    2900
2    2900
5    2900
4    2900
3    2900
0    2900
Name: count, dtype: int64

Total imágenes en entrenamiento: 69600
Total imágenes en validación: 17400


# ACTUAL MODEL

In [ ]:
# Parámetros
IMG_SIZE = (32, 32)
BATCH_SIZE = 64
EPOCHS = 3

label_map = {
    0: 'chat',
    1: 'email',
    2: 'voice',
    3: 'streaming',
    4: 'filetransfer',
    5: 'p2p'
}

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# MODELO

model = tf.keras.Sequential([
    # No usamos data aug porque no funciona bien
    layers.Rescaling(1./255),
    
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(32, 32, 1)),  # Cambiar 1 a 3 para 3 canales
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_map), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluación del modelo en el conjunto de validación
val_labels = np.array(val_df["label"])  # Etiquetas reales
predictions = model.predict(val_dataset)
pred_labels = np.argmax(predictions, axis=1)  # Etiquetas predichas

# Matriz de Confusión
conf_matrix = confusion_matrix(val_labels, pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Reporte de clasificación
print("\n📊 Classification Report:\n")
print(classification_report(val_labels, pred_labels, target_names=label_map.keys()))



NameError: name 'tf' is not defined